In [ ]:
# 한국어 위키 최신 dump 버전 다운로드
!wget https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-meta-current.xml.bz2

In [ ]:
# WikiExtractor 다운로드
!wget https://github.com/paul-hyun/web-crawler/raw/master/WikiExtractor.py

In [ ]:
# 현재 폴더 파일 목록 확인
os.listdir('.')

In [ ]:
# WikiExtractor 실행 (30분 이상 오랜 시간 소요 됨)
# -o: 출력할 폴더
# --json: json format으로 출력
os.system(f"python WikiExtractor.py -o kowiki --json kowiki-latest-pages-meta-current.xml.bz2")

In [ ]:
with open(os.path.join('kowiki', 'AA', 'wiki_00')) as f:
    for i, line in enumerate(f):
        if i >= 10:
            break
        line = line.strip()
        print(line)

In [ ]:
def list_wiki(dirname):
    """
    위키 목록을 읽어들임
    :param dirname: 위키 dir
    :return: 위키 파일 목록
    """
    filepaths = []
    filenames = os.listdir(dirname)
    for filename in filenames:
        filepath = os.path.join(dirname, filename)

        if os.path.isdir(filepath):
            filepaths.extend(list_wiki(filepath))
        else:
            find = re.findall(r"wiki_[0-9][0-9]", filepath)
            if 0 < len(find):
                filepaths.append(filepath)
    return sorted(filepaths)

In [ ]:
filepaths = []
dirnames = os.listdir('kowiki')
for dirname in dirnames:
    dirpath = os.path.join('kowiki', dirname)
    filenames = os.listdir(dirpath)
    for filename in filenames:
        if filename.startswith('wiki_'):
            filepath = os.path.join(dirpath, filename)
            filepaths.append(filepath)
filepaths = sorted(filepaths)
print(len(filepaths), filepaths[:10])

In [ ]:
def trim_text(item):
    """
    한 위키 문서 내의 여러줄띄기(\n\n...)를 한줄띄기로(\n)로 변경
    :param item: 위키 항목
    :return: text의 여러줄 new line을 한 줄 new line으로 변경한 json data
    """
    data = json.loads(item)
    text = data["text"]
    value = list(filter(lambda x: len(x) > 0, text.split('\n')))
    data["text"] = "\n".join(value)
    return data

In [ ]:
# 여러줄띄기(\n\n...)를 한줄띄기로(\n)로 변경
dataset = []
for filepath in tqdm(filepaths):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line:
                dataset.append(trim_text(line))
print(len(dataset))

In [ ]:
# 위키를 한 파일로 저장
with open("kowiki.txt", "w") as f:
    for data in tqdm(dataset):
        f.write(data["text"])
        f.write("\n\n\n\n")

In [ ]:
# 파일 내용 확인
with open("kowiki.txt") as f:
    for i, line in enumerate(f):
        if i >= 30:
            break
        line = line.strip()
        print(line)

In [ ]:
# 압축
!zip kowiki.txt.zip kowiki.txt

In [ ]:
# 압축파일 보관
shutil.move('kowiki.txt.zip', os.path.join(kowiki_dir, 'kowiki.txt.zip'))
os.listdir(kowiki_dir)